<a href="https://colab.research.google.com/github/cedamusk/AI-N-ML/blob/main/Svm_for_climate_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api geemap scikit-learn

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-climatechangeforedu')

In [ ]:
import geemap
import sklearn


In [ ]:
# Example script to load and visualize ERA5 climate reanalysis parameters in
# Google Earth Engine

# Daily mean 2m air temperature
era5_2mt = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_2m_air_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)
display(era5_2mt)

# Daily total precipitation sums
era5_tp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('total_precipitation')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean 2m dewpoint temperature
era5_2d = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('dewpoint_2m_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean sea-level pressure
era5_mslp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_sea_level_pressure')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean surface pressure
era5_sp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('surface_pressure')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean 10m u-component of wind
era5_u_wind_10m = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('u_component_of_wind_10m')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Convert pressure levels from Pa to hPa - Example for surface pressure
era5_sp = era5_sp.map(
    lambda image: image.divide(100).set(
        'system:time_start', image.get('system:time_start')
    )
)

# Visualization palette for total precipitation
vis_tp = {
    'min': 0.0,
    'max': 0.1,
    'palette': ['ffffff', '00ffff', '0080ff', 'da00ff', 'ffa400', 'ff0000'],
}

# Visualization palette for temperature (mean, min and max) and 2m dewpoint
# temperature
vis_2mt = {
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
}

# Visualization palette for u- and v-component of 10m wind
vis_wind = {
    'min': 0,
    'max': 30,
    'palette': [
        'ffffff',
        'ffff71',
        'deff00',
        '9eff00',
        '77b038',
        '007e55',
        '005f51',
        '004b51',
        '013a7b',
        '023aad',
    ],
}

# Visualization palette for pressure (surface pressure, mean sea level
# pressure) - adjust min and max values for mslp to 'min':990 and 'max':1050
vis_pressure = {
    'min': 500,
    'max': 1150,
    'palette': [
        '01ffff',
        '058bff',
        '0600ff',
        'df00ff',
        'ff00ff',
        'ff8c00',
        'ff8c00',
    ],
}


# Add layer to map
m = geemap.Map()
m.add_layer(
    era5_tp.filter(ee.Filter.date('2019-07-15')),
    vis_tp,
    'Daily total precipitation sums',
)
m.add_layer(
    era5_2d.filter(ee.Filter.date('2019-07-15')),
    vis_2mt,
    'Daily mean 2m dewpoint temperature',
)
m.add_layer(
    era5_2mt.filter(ee.Filter.date('2019-07-15')),
    vis_2mt,
    'Daily mean 2m air temperature',
)
m.add_layer(
    era5_u_wind_10m.filter(ee.Filter.date('2019-07-15')),
    vis_wind,
    'Daily mean 10m u-component of wind',
)
m.add_layer(
    era5_sp.filter(ee.Filter.date('2019-07-15')),
    vis_pressure,
    'Daily mean surface pressure',
)

m.set_center(21.2, 22.2, 2)
m

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import(classification_report, confusion_matrix, accuracy_score, f1_score)

In [ ]:
def preprocess_climate_data(image_collection, start_date, end_date):
  try:
    if not isinstance(image_collection, ee.ImageCollection):
      raise ValueError("Input must be an ee.ImageCollection")

    filtered_collection=image_collection.filter(
      ee.Filter.date(start_date, end_date)
    )

    if bands is None:
      bands=filtered_collection.first().bandNames().getInfo()

    reduced_image=filtered_collection.mean()

    selected_image=reduced_image.select(bands)

    data_array=selected_image.reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=ee.Geometry.Rectangle([-180, -90, 180, 90]),
        scale=1000
    )


  feature_array=np.array(filtered_collection.getInfo())
  return feature_array

In [ ]:
def create_climate_zone_labels(coordinates, climate_zones):
  labels=[]
  for coord in coordinates:
    for zone, zone_coords in climate_zones.items():
      if (zone_coords['lon_min']<= coord[0]<=zone_coords['lon_max']and
          zone_coords['lat_min']<=coord[1]<=zone_coords['lat_max']):
          labels.append(zone)
          break

    else:
      labels.append('Unknown')

  return np.array(labels)

In [ ]:
def train_climate_zone_svm(X,y):
  X_train, X_test, y_train, y_test=train_test_split(
      X, y, test_size=0.2, random_state=42
  )

  scaler=StandardScaler()
  X_train_scaled=scaler.fit_transform
  X_test_scaled=scaler.transform(X_test)

  svm_classifier=SVC(
      kernel='rbf',
      C=1.0,
      gamma='scale',
      random_state=42
  )
  svm_classifier.fit(X_train_scaled, y_train)

  y_pred=svm_classifier.predict(X_test_scaled)

  metrics={
      'accuracy': accuracy_score(y_test, y_pred),
      'classification_report': classification_report(y_test, y_pred),
      'confusion_matrix': confusion_matrix(y_test, y_pred)
  }

  return svm_classifier, scaler, metrics



In [ ]:
def main():
  climate_parameters=[
      'mean_2m_air_temperature',
      'total_precipitation',
      'dewpoint_2m_temperature',
      'mean_sea_level_pressure',
      'surface_pressure',
      'u_component_of_wind_10m'
  ]

  climate_zones={
      'Tropical':{'lon_min':-90, 'lon_max':0, 'lat_min':-30, 'lat_max':30},
      'Arid':{'lon_min': -20, 'lon_max':50, 'lat_min':10, 'lat_max':40},
      'Temperature':{'lon_min':-100, 'lon_max':40, 'lat_min':30, 'lat_max':60},
      'Polar':{'lon_min':-180, 'lon_max':180, 'lat_min':60, 'lat_max':90}
  }

  era5_data_collection=ee.ImageCollection('ECMWF/ERA5/DAILY')

  X=preprocess_climate_data(
      era5_data_collection,
      start_date='2019-01-01',
      end_date='2019-12-31',
      bands=climate_bands
  )

  print("Processed Data Shape:", processed_data.shape)
  print("Processed Data Sample:\n", processed_data[:5])

  coordinates=[[lon, lat] for lon in range(-100, 100, 10) for lat in range(-90, 90, 10)]
  y=create_climate_zone_labels(coordinates, climate_zones)

  svm_model, scaler, metrics=train_climate_zone_svm(X, y)

  print("SVM Climate Zone Classification results:")
  print(f"Accuracy: {metrics['accuracy']}")
  print("\nClassification Report:")
  print(metrics['classification_report'])
  print("\nConfusion Matrix:")
  print(metrics['confusion_matrix'])

  return svm_model, scaler, processed_data

In [ ]:
if __name__=='__main__':
  model, scaler=main()